In [1]:
!pip install datasets numpy pandas transformers[torch] wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [2]:
from datasets import Dataset, DatasetDict, load_metric
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DefaultDataCollator,
    TrainingArguments,
    EarlyStoppingCallback,
    Trainer,
)
import wandb

In [3]:
for lr in [1e-4, 1e-5, 5e-5]:
  for wd in [0, 0.01, 0.02]:
    for batch_size in [8, 16]:

      # Get model and tokenizer as global variables
      model_name = "FacebookAI/xlm-roberta-base"
      project_name = "XLM-RoBERTa-Gender-Bias-Detection"
      run_name = f"run_[lr]_[wl]_[bs]"
      tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

      model = AutoModelForSequenceClassification.from_pretrained(
          model_name,
          num_labels=2,
      )

      wandb.init(project=project_name, name=run_name)

      def tokenize_function(examples):
        return tokenizer(
            examples["text"], padding="max_length", max_length=100, truncation=True
        )

      def compute_metrics(eval_pred):
        metric1 = load_metric("precision")
        metric2 = load_metric("recall")
        metric3 = load_metric("f1")
        metric4 = load_metric("accuracy")

        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)

        precision = metric1.compute(
            predictions=predictions, references=labels, average="macro"
        )["precision"]
        recall = metric2.compute(
            predictions=predictions, references=labels, average="macro"
        )["recall"]
        f1 = metric3.compute(predictions=predictions, references=labels, average="macro")[
            "f1"
        ]
        accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

        return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

    # Loading data
    train_data = pd.read_csv("train_data.csv")
    test_data = pd.read_csv("test_data.csv")
    valid_data = pd.read_csv("val_data.csv")

    train_data = train_data.rename(columns={"Contents": "text"})
    test_data = test_data.rename(columns={"Contents": "text"})
    valid_data = valid_data.rename(columns={"Contents": "text"})

    train_data = train_data.rename(columns={"Label": "label"})
    test_data = test_data.rename(columns={"Label": "label"})
    valid_data = valid_data.rename(columns={"Label": "label"})

    # Drop all columns that are not "text" and "label"
    train_data = train_data[["text", "label"]]
    test_data = test_data[["text", "label"]]
    valid_data = valid_data[["text", "label"]]

    # Create a dataset dictionary
    dataset_dict = {
        "train": Dataset.from_pandas(train_data),
        "test": Dataset.from_pandas(test_data),
        "valid": Dataset.from_pandas(valid_data),
    }

    # Turn it into HuggingFace dataset
    dataset = DatasetDict(dataset_dict)

    # Tokenize the data
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset["train"][0]

        # Defining the Data Collator
    data_collator = DefaultDataCollator()

    # Training
    training_args = TrainingArguments(
        output_dir="output",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        weight_decay=0.01,
        report_to="wandb",
        metric_for_best_model="eval_f1",
        load_best_model_at_end=True,
        fp16=True,
    )

    trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["valid"],
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
            callbacks=[
                EarlyStoppingCallback(
                    early_stopping_patience=3, early_stopping_threshold=0.00
                )
            ],
        )

    trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

Map:   0%|          | 0/1339 [00:00<?, ? examples/s]

Map:   0%|          | 0/287 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.429462,0.414931,0.500000,0.453510,0.829861
2,No log,0.420650,0.867399,0.628469,0.666016,0.868056
3,No log,0.393386,0.671490,0.690248,0.679709,0.809028
4,No log,0.456095,0.658645,0.720092,0.674921,0.777778
5,No log,0.519079,0.676835,0.732645,0.694737,0.798611
6,0.324100,0.498968,0.676754,0.706473,0.688820,0.809028
7,0.324100,0.654009,0.702601,0.761421,0.722892,0.819444
8,0.324100,0.622484,0.728007,0.680813,0.699288,0.847222
9,0.324100,0.661283,0.709481,0.729229,0.718383,0.833333
10,0.324100,0.663799,0.722917,0.719281,0.721068,0.843750


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for recall contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/recall/recall.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric fro